In [2]:
import pandas as pd
from dateutil import parser

In [3]:
    df_reviews = pd.read_parquet('df_reviews_1.parquet')
    df_gastos_items = pd.read_parquet('df_gastos_items.parquet')   
    df_genre_ranking = pd.read_parquet('df_genre_ranking.parquet')
    df_playtime_forever = pd.read_parquet('df_playtime_forever.parquet')   

In [24]:
df_playtime_forever.head()

,genres,user_id,playtime_horas,user_url
0,"['Action', 'Adventure', 'Casual', 'Free to Pla...",12das,0.133333,http://steamcommunity.com/id/12das
1,"['Action', 'Adventure', 'Casual', 'Free to Pla...",666999661,15.550000,http://steamcommunity.com/id/666999661
2,"['Action', 'Adventure', 'Casual', 'Free to Pla...",76561197967469174,0.000000,http://steamcommunity.com/profiles/76561197967...
3,"['Action', 'Adventure', 'Casual', 'Free to Pla...",76561197983367658,0.000000,http://steamcommunity.com/profiles/76561197983...
4,"['Action', 'Adventure', 'Casual', 'Free to Pla...",76561197985797191,0.000000,http://steamcommunity.com/profiles/76561197985...


In [25]:
df_playtime_forever.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1704216 entries, 0 to 1704215
Data columns (total 4 columns):
 #   Column          Dtype  
---  ------          -----  
 0   genres          object 
 1   user_id         object 
 2   playtime_horas  float64
 3   user_url        object 
dtypes: float64(1), object(3)
memory usage: 52.0+ MB


In [11]:
df_games_etl_1 = pd.read_parquet('funcion_desarrollador', engine='pyarrow')
df_games_etl_1

,price,year,developer,item_id
0,4.99,2018,Kotoshiro,761140.0
1,0.00,2018,Secret Level SRL,643980.0
2,0.00,2017,Poolians.com,670290.0
3,0.99,2017,彼岸领域,767400.0
4,2.99,2016,0,773570.0
...,...,...,...,...
32127,1.99,2018,"Nikita ""Ghost_RUS""",773640.0
32128,4.99,2018,Sacada,733530.0
32129,1.99,2018,Laush Dmitriy Sergeevich,610660.0
32130,4.99,2017,"xropi,stev3ns",658870.0


In [13]:
df_etl_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5094105 entries, 0 to 5094104
Data columns (total 8 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   item_id           int64 
 1   item_name         object
 2   playtime_forever  int64 
 3   playtime_2weeks   int64 
 4   steam_id          int64 
 5   items_count       int64 
 6   user_id           object
 7   user_url          object
dtypes: int64(5), object(3)
memory usage: 310.9+ MB


In [4]:
developer("RewindApp")


{'cantidad_por_año': {2017: 8, 2018: 1},
 'porcentaje_gratis_por_año': {2017: 0, 2018: 100}}

In [85]:
def userdata(user_id):
    #Cargar df formato parquet
    df_reviews = pd.read_parquet('df_reviews_1.parquet')
    df_gastos_items = pd.read_parquet('df_gastos_items.parquet')   
    df_genre_ranking = pd.read_parquet('df_genre_ranking.parquet')
    df_playtime_forever = pd.read_parquet('df_playtime_forever.parquet')   

    try:
        # Filtra por el usuario de interés
        usuario = df_reviews[df_reviews['user_id'] == user_id]
        
        # Calcula la cantidad de dinero gastado para el usuario de interés
        cantidad_dinero = float(df_gastos_items[df_gastos_items['user_id'] == user_id]['price'].iloc[0])
        
        # Busca el count_items para el usuario de interés
        count_items = int(df_gastos_items[df_gastos_items['user_id'] == user_id]['items_count'].iloc[0])
        
        # Filtra las recomendaciones del usuario de interés
        recomendaciones_usuario = usuario['reviews_recommend']
        
        # Calcula el total de recomendaciones realizadas por el usuario de interés
        total_recomendaciones = recomendaciones_usuario[recomendaciones_usuario == 'Recomendado'].count()
        
        # Calcula el total de reviews realizada por todos los usuarios
        total_reviews = len(df_reviews['user_id'].unique())
        
        # Calcula el porcentaje de recomendaciones realizadas por el usuario de interés
        porcentaje_recomendaciones = (total_recomendaciones / total_reviews) * 100
        
        return {
            'cantidad_dinero': cantidad_dinero,
            'porcentaje_recomendacion': round(porcentaje_recomendaciones, 2),
            'total_items': count_items
        }
    except KeyError:
        return {
            'error': 'Usuario no encontrado en los DataFrames'
        }


In [86]:
userdata('js41637')

{'cantidad_dinero': 8553.07,
 'porcentaje_recomendacion': 0.0,
 'total_items': 888}

In [59]:
# Crear una nueva columna 'genres_list' que contendrá listas de géneros separados
df_playtime_forever['genres_list'] = df_playtime_forever['genres'].apply(lambda x: x.strip("[]").split(", "))

# Crear un DataFrame df_genres_separados con una fila por cada género
df_genres_separados = pd.DataFrame(df_playtime_forever['genres_list'].explode().unique(), columns=['genre'])

In [66]:
df_genres_separados

,genre
0,'Action'
1,'Adventure'
2,'Casual'
3,'Free to Play'
4,'Indie'
5,'Early Access'
6,'Massively Multiplayer'
7,'RPG'
8,'Racing'
9,'Simulation'


In [67]:
df_playtime_forever

,genres,user_id,playtime_horas,user_url,genres_list
0,"['Action', 'Adventure', 'Casual', 'Free to Pla...",12das,0.133333,http://steamcommunity.com/id/12das,"['Action', 'Adventure', 'Casual', 'Free to Pla..."
1,"['Action', 'Adventure', 'Casual', 'Free to Pla...",666999661,15.550000,http://steamcommunity.com/id/666999661,"['Action', 'Adventure', 'Casual', 'Free to Pla..."
2,"['Action', 'Adventure', 'Casual', 'Free to Pla...",76561197967469174,0.000000,http://steamcommunity.com/profiles/76561197967...,"['Action', 'Adventure', 'Casual', 'Free to Pla..."
3,"['Action', 'Adventure', 'Casual', 'Free to Pla...",76561197983367658,0.000000,http://steamcommunity.com/profiles/76561197983...,"['Action', 'Adventure', 'Casual', 'Free to Pla..."
4,"['Action', 'Adventure', 'Casual', 'Free to Pla...",76561197985797191,0.000000,http://steamcommunity.com/profiles/76561197985...,"['Action', 'Adventure', 'Casual', 'Free to Pla..."
...,...,...,...,...,...
1704211,['Web Publishing'],xjerseypsycho,0.000000,http://steamcommunity.com/id/xjerseypsycho,['Web Publishing']
1704212,['Web Publishing'],xxmaster01,0.000000,http://steamcommunity.com/id/xxmaster01,['Web Publishing']
1704213,['Web Publishing'],yuvii28,0.000000,http://steamcommunity.com/id/yuvii28,['Web Publishing']
1704214,['Web Publishing'],zachster,0.583333,http://steamcommunity.com/id/zachster,['Web Publishing']


In [72]:
# Mostrar los primeros registros de la columna "genres_list"
print(df_playtime_forever['genres_list'].head())

0    ['Action', 'Adventure', 'Casual', 'Free to Pla...
1    ['Action', 'Adventure', 'Casual', 'Free to Pla...
2    ['Action', 'Adventure', 'Casual', 'Free to Pla...
3    ['Action', 'Adventure', 'Casual', 'Free to Pla...
4    ['Action', 'Adventure', 'Casual', 'Free to Pla...
Name: genres_list, dtype: object


In [74]:
# Crear un nuevo DataFrame con géneros individuales
df_genres_separados = df_playtime_forever.explode('genres_list')

In [75]:
df_genres_separados

,genres,user_id,playtime_horas,user_url,genres_list
0,"['Action', 'Adventure', 'Casual', 'Free to Pla...",12das,0.133333,http://steamcommunity.com/id/12das,'Action'
0,"['Action', 'Adventure', 'Casual', 'Free to Pla...",12das,0.133333,http://steamcommunity.com/id/12das,'Adventure'
0,"['Action', 'Adventure', 'Casual', 'Free to Pla...",12das,0.133333,http://steamcommunity.com/id/12das,'Casual'
0,"['Action', 'Adventure', 'Casual', 'Free to Pla...",12das,0.133333,http://steamcommunity.com/id/12das,'Free to Play'
0,"['Action', 'Adventure', 'Casual', 'Free to Pla...",12das,0.133333,http://steamcommunity.com/id/12das,'Indie'
...,...,...,...,...,...
1704211,['Web Publishing'],xjerseypsycho,0.000000,http://steamcommunity.com/id/xjerseypsycho,'Web Publishing'
1704212,['Web Publishing'],xxmaster01,0.000000,http://steamcommunity.com/id/xxmaster01,'Web Publishing'
1704213,['Web Publishing'],yuvii28,0.000000,http://steamcommunity.com/id/yuvii28,'Web Publishing'
1704214,['Web Publishing'],zachster,0.583333,http://steamcommunity.com/id/zachster,'Web Publishing'


In [78]:
df_genres_separados.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5394603 entries, 0 to 1704215
Data columns (total 4 columns):
 #   Column          Dtype  
---  ------          -----  
 0   user_id         object 
 1   playtime_horas  float64
 2   user_url        object 
 3   genres_list     object 
dtypes: float64(1), object(3)
memory usage: 205.8+ MB


In [76]:
# Eliminar la columna "genres"
df_genres_separados.drop(columns=['genres'], inplace=True)

# Mostrar los primeros registros del DataFrame actualizado
print(df_genres_separados.head())

  user_id  playtime_horas                            user_url     genres_list
0   12das        0.133333  http://steamcommunity.com/id/12das        'Action'
0   12das        0.133333  http://steamcommunity.com/id/12das     'Adventure'
0   12das        0.133333  http://steamcommunity.com/id/12das        'Casual'
0   12das        0.133333  http://steamcommunity.com/id/12das  'Free to Play'
0   12das        0.133333  http://steamcommunity.com/id/12das         'Indie'


In [62]:
def genre(genero):
    '''
    Esta función devuelve la posición de un género de videojuego en un ranking basado en la cantidad de horas jugadas.
         
    Args:
        genero (str): Género del videojuego.
    
    Returns:
        dict: Un diccionario que contiene la posición del género en el ranking.
            - 'rank' (int): Posición del género en el ranking basado en las horas jugadas.
    '''
    # Busca el ranking para el género de interés
    rank = df_genre_ranking[df_genre_ranking['genres'] == genero]['ranking'].iloc[0]
    return {
        'rank': int(rank)
    }

In [77]:
# Guardar el DataFrame en formato Parquet
df_genres_separados.to_parquet('df_genres_separados.parquet', index=False)

In [101]:
 df_genre_ranking = pd.read_parquet('df_genre_ranking.parquet')

In [102]:
df_genre_ranking.info()

<class 'pandas.core.frame.DataFrame'>
Index: 588 entries, 262 to 342
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   genres          588 non-null    object 
 1   playtime_horas  588 non-null    float64
 2   ranking         588 non-null    int32  
dtypes: float64(1), int32(1), object(1)
memory usage: 16.1+ KB


In [105]:
def genre(genero):

    df_genre_ranking = pd.read_parquet('df_genre_ranking.parquet')
    
    try:
        # Busca el ranking para el género de interés
        rank = df_genre_ranking.loc[df_genre_ranking['genres'] == genero, 'ranking'].iloc[0]
    except IndexError:
        # Manejo del error: género no encontrado
        rank = None
    return {
        'rank': rank
    }

In [106]:
genero = 'Simulation'
genre(genero)

{'rank': None}

In [107]:
def userforgenre(genero):
    '''
    Esta función devuelve el top 5 de usuarios con más horas de juego en un género específico, junto con su URL de perfil y ID de usuario.
         
    Args:
        genero (str): Género del videojuego.
    
    Returns:
        dict: Un diccionario que contiene el top 5 de usuarios con más horas de juego en el género dado, junto con su URL de perfil y ID de usuario.
            - 'user_id' (str): ID del usuario.
            - 'user_url' (str): URL del perfil del usuario.
    '''
    # Filtra el dataframe por el género de interés
    data_por_genero = df_playtime_forever[df_playtime_forever['genres'].str.contains(genero)]

    # Agrupa el dataframe filtrado por usuario y suma la cantidad de horas
    top_users = data_por_genero.groupby(['user_url', 'user_id'])['playtime_horas'].sum().nlargest(5).reset_index()

    # Crear una lista de diccionarios con la información de los usuarios
    top_users_list = []
    for index, row in top_users.iterrows():
        user_info = {
            'user_id': row['user_id'],
            'user_url': row['user_url']
        }
        top_users_list.append(user_info)

    # Crear el diccionario de retorno
    result = {
        'Top 5 usuarios para el género ' + genero: top_users_list
    }
    return result

In [108]:
genero = 'Action'
userforgenre(genero)

{'Top 5 usuarios para el género Action': [{'user_id': 'Sp3ctre',
   'user_url': 'http://steamcommunity.com/id/Sp3ctre'},
  {'user_id': 'shinomegami',
   'user_url': 'http://steamcommunity.com/id/shinomegami'},
  {'user_id': 'REBAS_AS_F-T',
   'user_url': 'http://steamcommunity.com/id/REBAS_AS_F-T'},
  {'user_id': 'Terminally-Chill',
   'user_url': 'http://steamcommunity.com/id/Terminally-Chill'},
  {'user_id': 'DownSyndromeKid',
   'user_url': 'http://steamcommunity.com/id/DownSyndromeKid'}]}

In [10]:
df_reviews_1 = pd.read_parquet('df_reviews_2.parquet')
df_reviews_1

,user_id,user_url,reviews_item_id,reviews_helpful,reviews_recommend,reviews_date,sentiment_analysis,release_anio
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1250,No ratings yet,True,2011-11-05,1,2009
1,EndAtHallow,http://steamcommunity.com/id/EndAtHallow,1250,No ratings yet,True,2015-01-15,1,2009
2,76561198077432581,http://steamcommunity.com/profiles/76561198077...,1250,No ratings yet,True,2014-12-12,1,2009
3,76561198057958244,http://steamcommunity.com/profiles/76561198057...,1250,0 of 1 people (0%) found this review helpful,True,2013-12-13,0,2009
4,46366536564574576346346546,http://steamcommunity.com/id/46366536564574576...,1250,2 of 3 people (67%) found this review helpful,True,2014-08-19,1,2009
...,...,...,...,...,...,...,...,...
48797,ButtBurger2,http://steamcommunity.com/id/ButtBurger2,73010,No ratings yet,True,2012-05-17,0,2011
48798,76561198064526566,http://steamcommunity.com/profiles/76561198064...,378930,3 of 17 people (18%) found this review helpful,False,Formato inválido,1,2016
48799,haungaraho,http://steamcommunity.com/id/haungaraho,16600,No ratings yet,True,2012-10-22,2,2008
48800,UnseenPrecision,http://steamcommunity.com/id/UnseenPrecision,232950,No ratings yet,True,2014-01-19,1,2013


In [9]:
df_reviews.head()

,user_id,user_url,index,funny,posted,last_edited,item_id,helpful,recommend,review,sentiment,reviews_recommend
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,0,,2011-11-05,,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,2,Recomendado
1,js41637,http://steamcommunity.com/id/js41637,1,,2011-07-15,,22200,No ratings yet,True,It's unique and worth a playthrough.,2,Recomendado
2,evcentric,http://steamcommunity.com/id/evcentric,2,,2011-04-21,,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,2,Recomendado
3,doctr,http://steamcommunity.com/id/doctr,3,,2014-06-24,,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,2,Recomendado
4,maplemage,http://steamcommunity.com/id/maplemage,4,,2013-09-08,,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,2,Recomendado


In [12]:
def best_developer_of_year(fecha_inicio, fecha_fin):

    df_reviews_1 = pd.read_parquet('df_reviews_2.parquet')

    # Filtra el dataframe entre las fechas de interés
    user_data_entre_fechas = df_reviews_1[(df_reviews_1['reviews_date'] >= fecha_inicio) & (df_reviews_1['reviews_date'] <= fecha_fin)]
    # Calcula la cantidad de usuarios que dieron reviews entre las fechas de interés
    total_usuarios = user_data_entre_fechas['user_id'].nunique()
    # Calcula el total de recomendaciones entre las fechas de interes (True + False)
    total_recomendacion = len(user_data_entre_fechas)
    # Calcula la cantidad de recomendaciones positivas que que hicieron entre las fechas de interés
    total_recomendaciones_True = user_data_entre_fechas['reviews_recommend'].sum()
    # Calcula el porcentaje de recomendación realizadas entre el total de usuarios
    porcentaje_recomendaciones = (total_recomendaciones_True / total_recomendacion) * 100
    
    return {
        'total_usuarios_reviews': total_usuarios,
        'porcentaje_recomendaciones': round(porcentaje_recomendaciones,2)
    }


In [13]:
best_developer_of_year('2011-11-05', '2012-12-24')

{'total_usuarios_reviews': 822, 'porcentaje_recomendaciones': 98.73}